# How to learn feature for functional maps

In this notebook, we show how to use deep functional maps to learn feature for 3d shape matching.

In [1]:
import os

os.environ["GEOMSTATS_BACKEND"] = "pytorch"

import torch

from geomfum.convert import P2pFromFmConverter, TorchNeighborFinder
from geomfum.dataset.torch import PairsDataset, ShapeDataset
from geomfum.descriptor.learned import FeatureExtractor
from geomfum.forward_functional_map import ForwardFunctionalMap
from geomfum.learning.losses import (
    BijectivityLoss,
    GeodesicError,
    LaplacianCommutativityLoss,
    LossManager,
    OrthonormalityLoss,
)
from geomfum.learning.models import FMNet
from geomfum.learning.trainer import DeepFunctionalMapTrainer

First, we define our model. We can instantiate it combining feature extractors and forward logic, however, we provide some classic frameworks, like FMNet.

In [2]:
# Build the model
fmap_module = ForwardFunctionalMap(1e3, 1, True)

feature_extractor = FeatureExtractor.from_registry(
    which="diffusionnet",
    device="cuda",
    k_eig=200,
)

functional_map_model = FMNet(
    feature_extractor=feature_extractor,
    fmap_module=fmap_module,
    converter=P2pFromFmConverter(TorchNeighborFinder.from_registry(which="densemap")),
)


/home/ubuntu/giulio_vigano/geomfum_proj/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Then, we instantiate the training dataset and we split it for training purposes.

In [3]:
# TRAIN_SET_PATH = "../../../datasets/faust/train_set/"
# dataset = ShapeDataset(
#    TRAIN_SET_PATH, spectral=True, distances=False, device="cuda", k=30
# )
# train_size = int(0.8 * len(dataset))
# val_size = len(dataset) - train_size

# train_shapes, validation_shapes = random_split(dataset, [train_size, val_size])
# train_dataset = PairsDataset(
#    train_shapes,
#    pair_mode="all",
# )

# validation_dataset = PairsDataset(
#    validation_shapes,
#    pair_mode="all",
# )


Since a lot of time we do not need to store distances for training shapes, but they are usefull for validation shapes, we can do the following trick

In [ ]:
# build the train and test loaders
TRAIN_SET_PATH = "../../../datasets/smal/train_set/"
TEST_SET_PATH = "../../../datasets/smal/test_set/"
# instantiate
# the full dataset with default attributes
train_shapes = ShapeDataset(
    TRAIN_SET_PATH,
    spectral=True,
    distances=False,
    device="cuda",
    k=30,
)
validation_shapes = ShapeDataset(
    TEST_SET_PATH,
    spectral=True,
    distances=True,
    device="cuda",
    k=30,
)
train_dataset = PairsDataset(
    train_shapes,
    pair_mode="all",
)

validation_dataset = PairsDataset(
    validation_shapes,
    pair_mode="all",
)


/home/ubuntu/giulio_vigano/geomfum_proj/geomfum/geomfum/_backend/pytorch/sparse.py:22: UserWarning: Sparse CSC tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at /pytorch/aten/src/ATen/SparseCsrTensorImpl.cpp:53.)
  return _torch.sparse_csc_tensor(ccol_indices, row_indices, values, size=array.shape)


we build optimizer

In [ ]:
optimizer = torch.optim.Adam(functional_map_model.parameters(), lr=1e-3)

In [ ]:
import torch.nn as nn


class GeodesicError(nn.Module):
    """Computes the accuracy of a correspondence by measuring the mean of the geodesic distances between points of the predicted permuted target and the ground truth target."""

    def __init__(self):
        super().__init__()

    required_inputs = [
        "p2p12",
        "dist_a",
        "corr_a",
        "corr_b",
    ]

    def _compute_geodesic_loss(self, p2p, source_dist, source_corr, target_corr):
        """Compute the geodesic loss for batched inputs."""
        return torch.mean(source_dist[p2p[target_corr], source_corr])

    def forward(self, p2p12, dist_a, corr_a, corr_b):
        """Forward pass."""
        loss = self._compute_geodesic_loss(p2p12, dist_a, corr_a, corr_b)
        return loss


Now we define the losses that we will cnsider. Again we can define our own losses, howver we provide some classic functional map energies, like the orthonormality loss.

In [ ]:
# define the loss
losses = [
    OrthonormalityLoss(weight=1.0),
    BijectivityLoss(weight=1.0),
    LaplacianCommutativityLoss(weight=1e-3),
]
loss_manager = LossManager(losses)

losses = [
    GeodesicError(),
]

val_loss_manager = LossManager(losses)

We have defined a trainer for simplicity that thakes as input model, losses, train and val datasets and optimizer and manages the training loops.

In [ ]:
trainer = DeepFunctionalMapTrainer(
    model=functional_map_model,
    train_loss_manager=loss_manager,
    val_loss_manager=val_loss_manager,
    train_set=train_dataset,
    val_set=validation_dataset,
    optimizer=optimizer,
    device="cuda",
    epochs=10,
)

In [ ]:
trainer.train()

In [ ]:
trainer.validate()

In [ ]:
pair = trainer.val_set[8]  # Access item by index
trainer.optimizer.zero_grad()
trainer.model.eval()  # Set the model to training mode
mesh_a = pair["source"]["mesh"]
mesh_b = pair["target"]["mesh"]
with torch.no_grad():
    outputs = trainer.model(mesh_a, mesh_b)


In [ ]:
mask = trainer.model.fmap_module._compute_mask(mesh_a.basis.vals, mesh_b.basis.vals, 1)

In [ ]:
outputs["p2p12"]

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(outputs["fmap21"].detach().cpu().numpy())

In [ ]:
a, b = mesh_a.laplacian.find_spectrum(
    spectrum_size=30, set_as_basis=True, recompute=True
)
mesh_a.basis.vecs.shape
mesh_a.basis.full_vecs.shape

In [ ]:
a, b = mesh_a.laplacian.find_spectrum(
    spectrum_size=200, set_as_basis=False, recompute=True
)
mesh_a.basis.vecs.shape
mesh_a.basis.full_vecs.shape

In [ ]:
mesh_a.basis.vecs.shape

In [ ]:
mesh_a.laplacian.find_spectrum(spectrum_size=200, set_as_basis=False, recompute=False)